#### Calculate the final balance for each account after applying all deposits and withdrawals.

In [83]:
import pandas as pd
import sqlite3
import sql
sql.style = 'plain'
%load_ext sql
connection=sqlite3.connect("question_bank.db")
print("Successfully connected to SQL database")
# Connect to SQLite
%sql sqlite:///question_bank.db
print("success")

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
Successfully connected to SQL database
success


| transaction\_id | account\_id | transaction\_type | amount |
| --------------- | ----------- | ----------------- | ------ |
| 1               | 101         | Deposit           | 500    |
| 2               | 101         | Withdrawal        | 200    |
| 3               | 102         | Deposit           | 1000   |
| 4               | 101         | Deposit           | 100    |
| 5               | 102         | Withdrawal        | 300    |


* You need to calculate the final balance for each account after applying all deposits and withdrawals.

## Method 1

In [84]:
%%sql

with temp as (
    select
        account_id,
        case when transaction_type='Deposit' then amount end money_deposited,
        case when transaction_type='Withdrawal' then amount end money_withdrawn
    from transactions
),
temp_2 as (
    select 
        account_id,
        sum(money_deposited) as md,
        sum(money_withdrawed) as mw
    from temp
    group by account_id 
)
select *,(md-mw) as diff
from temp_2

 * sqlite:///question_bank.db
(sqlite3.OperationalError) no such column: account_id
[SQL: with temp as (
    select
        account_id,
        case when transaction_type='Deposit' then amount end money_deposited,
        case when transaction_type='Withdrawal' then amount end money_withdrawn
    from transactions
),
temp_2 as (
    select 
        account_id,
        sum(money_deposited) as md,
        sum(money_withdrawed) as mw
    from temp
    group by account_id 
)
select *,(md-mw) as diff
from temp_2]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


## Method 2

In [85]:
%%sql
select * from transactions

 * sqlite:///question_bank.db
Done.


transaction_id,sender_id,receiver_id,amount,transaction_date
1,1,2,1200,2025-08-17
2,2,3,6000,2025-08-22
3,3,4,800,2025-08-12
4,4,5,2000,2025-07-18
5,5,6,7000,2025-08-25
6,6,7,500,2025-08-26


In [86]:
%%sql

select
    account_id,
    sum(case when transaction_type='Deposit' then amount else 0 end) as dtx,
    sum(case when transaction_type='Withdrawal' then amount else 0 end) as wtx,
    sum(case when transaction_type='Deposit' then amount else 0 end) - 
        sum(case when transaction_type='Withdrawal' then amount else 0 end) as diff
from transactions
group by account_id

 * sqlite:///question_bank.db
(sqlite3.OperationalError) no such column: account_id
[SQL: select
    account_id,
    sum(case when transaction_type='Deposit' then amount else 0 end) as dtx,
    sum(case when transaction_type='Withdrawal' then amount else 0 end) as wtx,
    sum(case when transaction_type='Deposit' then amount else 0 end) - 
        sum(case when transaction_type='Withdrawal' then amount else 0 end) as diff
from transactions
group by account_id]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


Question 2: Compute the average transaction amount per user.
* Rank users based on their average transaction amount (highest first).
* Handle ties properly (i.e., if two users have the same average, they should get the same rank).

In [87]:
%%sql
select * from transactions

 * sqlite:///question_bank.db
Done.


transaction_id,sender_id,receiver_id,amount,transaction_date
1,1,2,1200,2025-08-17
2,2,3,6000,2025-08-22
3,3,4,800,2025-08-12
4,4,5,2000,2025-07-18
5,5,6,7000,2025-08-25
6,6,7,500,2025-08-26


In [88]:
%%sql

with temp as(
    select 
        account_id,
        round(avg(amount),2) as average
    from transactions
    group by account_id
)
select 
    account_id,
    average,
    dense_rank() over (order by average desc) as rk
from temp

 * sqlite:///question_bank.db
(sqlite3.OperationalError) no such column: account_id
[SQL: with temp as(
    select 
        account_id,
        round(avg(amount),2) as average
    from transactions
    group by account_id
)
select 
    account_id,
    average,
    dense_rank() over (order by average desc) as rk
from temp]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


| Clause / Function                                             | When it Runs          | What it Does                                   | Example                              | Key Rule                                             |
| ------------------------------------------------------------- | --------------------- | ---------------------------------------------- | ------------------------------------ | ---------------------------------------------------- |
| **WHERE**                                                     | Before grouping       | Filters individual rows                        | `WHERE salary > 50000`               | Cannot use aggregates here                           |
| **GROUP BY**                                                  | After WHERE           | Groups rows by column(s)                       | `GROUP BY department`                | Every column in SELECT must be grouped or aggregated |
| **Aggregate Functions** (`SUM`, `AVG`, `COUNT`, `MIN`, `MAX`) | During grouping       | Collapse rows into single values               | `AVG(salary)`                        | Allowed only with grouped columns                    |
| **HAVING**                                                    | After GROUP BY        | Filters groups                                 | `HAVING AVG(salary) > 60000`         | Use for aggregate-based conditions                   |
| **Window Functions** (`SUM() OVER`, `RANK() OVER`)            | After GROUP BY/HAVING | Add extra calculations without collapsing rows | `RANK() OVER (ORDER BY salary DESC)` | Keep row-level detail, don’t require GROUP BY        |


## ----------------

| ✅ Allowed in SELECT                         | ❌ Not Allowed in SELECT                       |
| ------------------------------------------- | --------------------------------------------- |
| - Columns listed in `GROUP BY`              | - Raw columns not in `GROUP BY` or aggregates |
| - Aggregates: `SUM`, `AVG`, `COUNT`, `MIN`… | - `SELECT *` with aggregation                 |


In [89]:
%%sql

select
    account_id,
    sum(case 
            when transaction_type='Deposit' then amount 
            when transaction_type='Withdrawal' then - amount
            else 0
    end) as dff
from transactions
group by account_id

 * sqlite:///question_bank.db
(sqlite3.OperationalError) no such column: account_id
[SQL: select
    account_id,
    sum(case 
            when transaction_type='Deposit' then amount 
            when transaction_type='Withdrawal' then - amount
            else 0
    end) as dff
from transactions
group by account_id]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
